In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
import tensorflow as tf
import sys
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pickle
from statsmodels.regression.quantile_regression import QuantReg
import random

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
def split_data(data, test_ratio):
    def make2dList(rows, cols):
        a=[]
        for row in range(rows): a += [[0]*cols]
        return a
    test_size = int(len(data) * test_ratio)
    training_data = make2dList(len(data) - test_size, data[0]["quantile"].shape[0] * data[0]["quantile"].shape[1])
    training_labels = [0]*(len(data)-test_size)
    testing_data = make2dList(test_size, data[0]["quantile"].shape[0] * data[0]["quantile"].shape[1])
    testing_labels = [0]*test_size
    
    test = random.sample(range(0, len(data)), test_size)
    
    start = -1
    for key in range(len(data)):
        if (key in test): continue
        start += 1
        for j in range(data[0]["quantile"].shape[0]):
            for k in range(data[0]["quantile"].shape[1]):
                training_data[start][k*data[0]["quantile"].shape[0] + j] = data[key]["quantile"][j][k]
        training_labels[start] = data[key]["class"]
    
    start = -1
    for key in test:
        start += 1
        for j in range(data[0]["quantile"].shape[0]):
            for k in range(data[0]["quantile"].shape[1]):
                testing_data[start][k*data[0]["quantile"].shape[0] + j] = data[key]["quantile"][j][k]
        testing_labels[start] = data[key]["class"]
        
    
    return training_data, training_labels, testing_data, testing_labels

In [3]:
from sklearn import svm

clf = svm.SVC(C = 1000)
data = pickle.load(open('../data/Quantiles.pickle','rb'))
def iteration(model):
    training_data, training_labels, testing_data, testing_labels = split_data(data, 0.1)
    model.fit(training_data, training_labels)
    pred = model.predict(testing_data)
    return (np.count_nonzero(pred == np.array(testing_labels)) / len(testing_labels))




In [75]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(clf, 100, algorithm = 'SAMME')

avg = 0
num = 50
for i in range(num):
    avg += iteration(clf)
print(avg / num)

0.8500379746835466
